In [ ]:
# File: cards.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

app = App.create()

mesh_res, n_stack, card_height = 8, 8, 0.25
card_width = 0.75 * card_height
V, F = app.mesh.rectangle(mesh_res, card_width, card_height, [0, 0, 1], [0, 1, 0])
app.asset.add.tri("card", V, F)

V, F, T = app.mesh.icosphere(r=0.15, subdiv_count=3).tetrahedralize()
app.asset.add.tet("sphere", V, F, T)

scene = app.scene.create("house-of-cards")
param = app.session.param()
angle, gap = 25.0, param.get("contact-ghat")


def make_row(n: int, _x: float, y: float) -> float:
    x, _y, ceil_x = 0, 0, []
    for i in range(n):
        left = scene.add("card").rotate(-angle, "z")
        left.at(x - left.min("x") + (_x if i == 0 else 0), y - left.min("y"), 0)
        if i == 0:
            _x = left.max("x")
        right = scene.add("card").rotate(angle, "z")
        shift = gap + left.max("x") - right.min("x")
        right.at(shift, y - right.min("y"), 0)
        if i < n - 1:
            ceil_x.append(right.max("x"))
        x = right.max("x") + gap
        max_y = right.max("y") + gap
    for i, x in enumerate(ceil_x):
        z = max_y if i % 2 == 0 else max_y + gap
        ceil = scene.add("card").rotate(-90, "z").at(x, z, 0)
        _y = max(_y, ceil.max("y"))
    return _x, _y + gap


_x, _y = -0.75, gap
for i in reversed(range(n_stack)):
    _x, _y = make_row(i + 1, _x, _y)

scene.add("sphere").at(-2, 1, 0).jitter().velocity(2.3, 0, 0)
scene.add.invisible.wall([0, 0, 0], [0, 1, 0])

fixed = scene.build().report()
fixed.preview()

In [ ]:
(
    param.set("friction", 0.5)
    .set("area-young-mod", 30000)
    .set("bend", 1e6)
    .set("dt", 0.01)
    .set("min-newton-steps", 32)
)
session = app.session.create(fixed)
session.start(param).preview(options={"wireframe": True})
session.stream()

In [ ]:
session.animate(options={"wireframe": True})

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()